In [1]:
from dask.distributed import Client, progress

from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=20)
cluster

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:   tcp://10.32.16.73:43647
distributed.scheduler - INFO -   dashboard at:                     :8787


In [2]:
len(cluster.workers)

20

In [3]:
import zarr
import netCDF4
import pytide
import xarray as xr
import numpy as np
import dask

In [4]:
import gcsfs
fs = gcsfs.GCSFileSystem('pangeo-181919', requester_pays=True)


In [5]:
%%time
zmap = fs.get_mapper('pangeo-meom/eNATL60-BLBT02-SSH-1h')
ds = xr.open_zarr(zmap)
ds_sorted=ds.sortby('time_counter')
ssh=ds_sorted['sossheig']
time=ds_sorted['time_counter']

distributed.scheduler - INFO - Register tcp://10.32.11.7:46695
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.11.7:46695
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.11.6:42037
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.11.6:42037
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.7.7:43405
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.7.7:43405
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.7.8:46855
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.7.8:46855
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.17.6:39979
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.17.6:39979
distributed.core - INFO - S

CPU times: user 1.7 s, sys: 196 ms, total: 1.9 s
Wall time: 6.4 s


In [6]:
t=time.values

In [7]:
wt = pytide.WaveTable()
wt = pytide.WaveTable(["M2", "S2", "N2", "O1", "K1"])

In [8]:
%%time
f, vu = wt.compute_nodal_modulations(t)

CPU times: user 10.7 ms, sys: 101 µs, total: 10.8 ms
Wall time: 9.76 ms


In [9]:
def dask_array_rechunk(da, axis=0):
    """Search for the optimal block cutting without modifying the axis 'axis'
    in order to optimize its access in memory."""
    nblocks = 1
    
    def calculate_chuncks_size(chunks, size):
        result = np.array(chunks).prod() * size
        return result / (1000**2)
       
    while True:
        chunks = []
        div = int(np.sqrt(nblocks))
        for index, item in enumerate(da.chunks):
            chunks.append(np.array(item).sum() * (div if index == axis else 1))
        chunks = tuple(item // div for index, item in enumerate(chunks))
        chuncks_size = calculate_chuncks_size(chunks, da.dtype.itemsize)
        if chuncks_size > 100 and chuncks_size < 150:
            return chunks
        nblocks += 1

In [10]:
ssh_rechunk = ssh.chunk(dask_array_rechunk(ssh))

In [11]:
def _apply_along_axis(arr, func1d, func1d_axis, func1d_args, func1d_kwargs):
    """Wrap apply_along_axis"""
    return np.apply_along_axis(func1d, func1d_axis, arr, *func1d_args,
                                  **func1d_kwargs)


def apply_along_axis(func1d, axis, arr, *args, **kwargs):
    """Apply the harmonic analysis to 1-D slices along the given axis."""
    arr = dask.array.core.asarray(arr)

    # Validate and normalize axis.
    arr.shape[axis]
    axis = len(arr.shape[:axis])

    # Rechunk so that analyze is applied over the full axis.
    arr = arr.rechunk(arr.chunks[:axis] + (arr.shape[axis:axis + 1], ) +
                      arr.chunks[axis + 1:])

    # Test out some data with the function.
    test_data = np.ones(args[0].shape[1], dtype=arr.dtype)
    test_result = np.array(func1d(test_data, *args, **kwargs))

    # Map analyze over the data to get the result
    # Adds other axes as needed.
    result = arr.map_blocks(
        _apply_along_axis,
        name=dask.utils.funcname(func1d) + '-along-axis',
        dtype=test_result.dtype,
        chunks=(arr.chunks[:axis] + test_result.shape + arr.chunks[axis + 1:]),
        drop_axis=axis,
        new_axis=list(range(axis, axis + test_result.ndim, 1)),
        func1d=func1d,
        func1d_axis=axis,
        func1d_args=args,
        func1d_kwargs=kwargs,
    )

    return result

In [12]:
future = apply_along_axis(pytide.WaveTable.harmonic_analysis, 0, ssh_rechunk,
                          *(f, vu))

In [ ]:
analysis = future.compute()